In [2]:
from time import sleep
from tqdm import tqdm

import numpy as np
import pandas as pd
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials

from keys import GOOG_API_KEY

In [3]:
## Load dataframes
# Polling averages
pres = pd.read_csv('averages/presidential_gen_approval.csv')
pres_issue = pd.read_csv('averages/presidential_issue_approvals.csv')
pres_rv = pd.read_csv('averages/presidential_RV_approval.csv')
generic_ballot = pd.read_csv('averages/generic_ballot.csv')

# Polls display tables
pres_polls = pd.read_csv('transformed_tables/gen_approval_polls.csv')
issue_polls = pd.read_csv('transformed_tables/issue_approval_polls.csv')
gb_polls = pd.read_csv('transformed_tables/generic_ballot_polls_disp.csv')

In [4]:
# Define the scope
scopes = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive"
]

# Authenticate with the service account
credentials = Credentials.from_service_account_file(
    GOOG_API_KEY,
    scopes=scopes
)

# Authorize gspread
client = gspread.authorize(credentials)

## Polling Averages

In [5]:
spreadsheet = client.open("snoutcounter-data")
worksheet = spreadsheet.worksheet("approval-stats")
worksheet.clear()

set_with_dataframe(worksheet, pres.reset_index())

In [6]:
worksheet = spreadsheet.worksheet("approval_stats_RV")
worksheet.clear()

set_with_dataframe(worksheet, pres_rv.reset_index())

In [7]:
issues = ['economy', 'immigration', 'inflation', 'trade_tariffs', 'foreign_policy', 'healthcare', 'crime']
issue_suffixes = ['econ', 'imm', 'infl', 'trade', 'fp', 'healthcare', 'crime']

for iss, suf in tqdm(zip(issues, issue_suffixes)):
    worksheet = spreadsheet.worksheet(f"approval_stats_{suf}")
    worksheet.clear()

    issue_df = pres_issue[pres_issue['issue'] == iss]
    set_with_dataframe(worksheet, issue_df.reset_index())

7it [00:06,  1.03it/s]


In [8]:
## Net issue approval
net_issues = pd.pivot_table(pres_issue[pres_issue['issue'].isin(issues)], 
                            values=['net', 'net_lower_ci', 'net_upper_ci'], index=['end_date'], columns=['issue'],
                           aggfunc="first").reset_index()
net_issues.columns = ['_'.join(col).strip() for col in net_issues.columns.values]
net_issues.head()

,end_date_,net_crime,net_economy,net_foreign_policy,net_healthcare,net_immigration,net_inflation,net_trade_tariffs,net_lower_ci_crime,net_lower_ci_economy,...,net_lower_ci_immigration,net_lower_ci_inflation,net_lower_ci_trade_tariffs,net_upper_ci_crime,net_upper_ci_economy,net_upper_ci_foreign_policy,net_upper_ci_healthcare,net_upper_ci_immigration,net_upper_ci_inflation,net_upper_ci_trade_tariffs
0,2025-01-21,0.0,7.963136,3.963136,0.000000,9.963136,-4.036864,-0.036864,0.0,7.963136,...,9.963136,-4.036864,-0.036864,0.0,7.963136,3.963136,0.000000,9.963136,-4.036864,-0.036864
1,2025-01-22,0.0,7.963136,3.963136,0.000000,9.963136,-4.036864,-0.036864,0.0,7.963136,...,9.963136,-4.036864,-0.036864,0.0,7.963136,3.963136,0.000000,9.963136,-4.036864,-0.036864
2,2025-01-23,0.0,2.892959,3.963136,-13.605594,3.553189,-4.036864,-0.036864,0.0,-4.083128,...,-5.266295,-4.036864,-0.036864,0.0,9.869046,3.963136,-13.605594,12.372673,-4.036864,-0.036864
3,2025-01-24,0.0,2.892959,3.963136,-13.605594,3.553189,-4.036864,-0.036864,0.0,-4.083128,...,-5.266295,-4.036864,-0.036864,0.0,9.869046,3.963136,-13.605594,12.372673,-4.036864,-0.036864
4,2025-01-25,0.0,2.892959,3.963136,-13.605594,3.553189,-4.036864,-0.036864,0.0,-4.083128,...,-5.266295,-4.036864,-0.036864,0.0,9.869046,3.963136,-13.605594,12.372673,-4.036864,-0.036864


In [9]:
worksheet = spreadsheet.worksheet("net_approval_issues")
worksheet.clear()

set_with_dataframe(worksheet, net_issues.reset_index())

In [10]:
worksheet = spreadsheet.worksheet("generic_ballot_stats")
worksheet.clear()

set_with_dataframe(worksheet, generic_ballot.reset_index())

## Polls Display Tables

In [11]:
pres_polls['sponsors'] = '^Sponsor: ' + pres_polls['sponsors'] + '^'
pres_polls['sponsors'] = pres_polls['sponsors'].fillna('')
pres_polls = pres_polls.sort_values(['total_weight'], ascending=False)
pres_polls = pres_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
pres_polls['Sample'] = pres_polls['sample_size'].astype(int).astype(str) + ' ' + pres_polls['population']
pres_polls['Start Date'] = pd.to_datetime(pres_polls['Start Date']).astype(str)
pres_polls['End Date'] = pres_polls['End Date'].astype(str)
pres_polls = pres_polls.reset_index()
pres_polls['Pollster'] = ('<a href="' + pres_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          pres_polls['Pollster'] + '</u></a>' + pres_polls['sponsors'])
pres_polls = pres_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
pres_polls

,Pollster,Start Date,End Date,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://angusreid.org/one-year-of-don...",2026-01-16,2026-01-20,1838 A,1.177281e-01,37.0,56.0,-19.0,-16.978531
1,"<a href=""https://napolitannews.org/jobapproval...",2026-01-14,2026-01-21,3000 RV,9.606896e-02,47.0,51.0,-4.0,-13.840744
2,"<a href=""https://www.nytimes.com/2026/01/22/us...",2026-01-12,2026-01-17,1625 RV,8.818175e-02,40.0,56.0,-16.0,-15.426912
3,"<a href=""https://emersoncollegepolling.com/jan...",2026-01-17,2026-01-19,1000 LV,8.238218e-02,43.2,51.3,-8.1,-13.946780
4,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-16,2026-01-22,2226 A,5.535253e-02,38.0,58.0,-20.0,-18.048165
...,...,...,...,...,...,...,...,...,...
538,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,6.597106e-20,52.0,43.0,9.0,5.627514
539,"<a href=""https://www.ipsos.com/en-us/reuters-i...",2025-01-20,2025-01-21,1077 A,5.813607e-20,47.0,41.0,6.0,9.993947
540,"<a href=""https://substack.com/inbox/post/15539...",2025-01-21,2025-01-21,742 A,5.098023e-20,49.0,36.0,13.0,8.386435
541,"<a href=""https://quantusinsights.org/f/februar...",2025-02-01,2025-02-03,1000 RV,4.974562e-20,51.9,45.3,6.6,0.104328


In [12]:
worksheet = spreadsheet.worksheet("polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, pres_polls)

In [13]:
issue_polls['sponsor'] = '^Sponsor: ' + issue_polls['sponsor'] + '^'
issue_polls['sponsor'] = issue_polls['sponsor'].fillna('')
issue_polls = issue_polls.sort_values(['issue', 'total_weight'], ascending=False)
issue_polls = issue_polls.rename({
    'pollster':'Pollster', 'approve':'Approve', 'disapprove':'Disapprove', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net',
    'issue':'Issue'}, axis='columns')
issue_polls['Sample'] = issue_polls['sample_size'].astype(int).astype(str) + ' ' + issue_polls['population']
issue_polls['Start Date'] = pd.to_datetime(issue_polls['Start Date']).astype(str)
issue_polls['End Date'] = issue_polls['End Date'].astype(str)
issue_polls = issue_polls.reset_index()
issue_polls['Pollster'] = ('<a href="' + issue_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          issue_polls['Pollster'] + '</u></a>' + issue_polls['sponsor'])
issue_polls = issue_polls[issue_polls['Issue'].isin(issues)]
issue_polls['Issue'] = issue_polls['Issue'].replace({
    'economy': 'Economy',
    'immigration': 'Immigration',
    'foreign_policy': 'Foreign policy',
    'inflation': 'Inflation/prices',
    'trade_tariffs': 'Trade/tariffs',
    'healthcare': 'Healthcare',
    'crime': 'Crime'
})
issue_polls = issue_polls[['Pollster', 'Start Date', 'End Date', 'Issue', 'Sample', 'Weight', 'Approve', 'Disapprove', 'Net', 'Adjusted Net']]
issue_polls

,Pollster,Start Date,End Date,Issue,Sample,Weight,Approve,Disapprove,Net,Adjusted Net
0,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-16,2026-01-22,Trade/tariffs,2226 A,2.167270e-02,34.0,57.0,-23.0,-24.412029
1,"<a href=""https://pro.morningconsult.com/tracke...",2026-01-16,2026-01-18,Trade/tariffs,2201 RV,1.451660e-02,45.0,46.0,-1.0,-13.635653
2,"<a href=""https://pro.morningconsult.com/tracke...",2026-01-09,2026-01-12,Trade/tariffs,2201 RV,9.159356e-03,43.0,48.0,-5.0,-17.635653
3,"<a href=""https://s3.documentcloud.org/document...",2026-01-09,2026-01-12,Trade/tariffs,1209 A,8.413928e-03,37.0,62.0,-25.0,-19.717967
4,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-09,2026-01-12,Trade/tariffs,1602 A,7.336837e-03,37.0,56.0,-19.0,-20.412029
...,...,...,...,...,...,...,...,...,...,...
1066,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-23,2025-02-25,Crime,1604 A,4.323563e-14,47.0,37.0,10.0,8.587971
1067,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-16,2025-02-18,Crime,1603 A,2.502066e-14,48.0,37.0,11.0,9.587971
1068,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-09,2025-02-11,Crime,1595 A,1.450343e-14,45.0,41.0,4.0,2.587971
1069,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2025-02-02,2025-02-04,Crime,1604 A,8.498833e-15,48.0,35.0,13.0,11.587971


In [14]:
worksheet = spreadsheet.worksheet("polls_display_issues")
worksheet.clear()

set_with_dataframe(worksheet, issue_polls)

In [15]:
gb_polls['sponsors'] = '^Sponsor: ' + gb_polls['sponsors'] + '^'
gb_polls['sponsors'] = gb_polls['sponsors'].fillna('')
gb_polls = gb_polls.sort_values(['total_weight'], ascending=False)
gb_polls = gb_polls.rename({
    'pollster':'Pollster', 'dem':'Democrats', 'rep':'Republicans', 'net':'Net', 
    'total_weight':'Weight', 'start_date':'Start Date', 'end_date':'End Date', 'url':'URL', 'adj_net':'Adjusted Net'}, axis='columns')
gb_polls['Sample'] = gb_polls['sample_size'].astype(int).astype(str) + ' ' + gb_polls['population']
gb_polls['Start Date'] = pd.to_datetime(gb_polls['Start Date']).astype(str)
gb_polls['End Date'] = gb_polls['End Date'].astype(str)
gb_polls = gb_polls.reset_index()
gb_polls['Pollster'] = ('<a href="' + gb_polls['URL'] + '" style="color:black; text-decoration:underline;"><u>' + 
                          gb_polls['Pollster'] + '</u></a>' + gb_polls['sponsors'])
gb_polls = gb_polls[['Pollster', 'Start Date', 'End Date', 'Sample', 'Weight', 'Democrats', 'Republicans', 'Net', 'Adjusted Net']]
gb_polls

,Pollster,Start Date,End Date,Sample,Weight,Democrats,Republicans,Net,Adjusted Net
0,"<a href=""https://www.nytimes.com/2026/01/22/us...",2026-01-12,2026-01-17,1625 RV,1.522621e-01,48.0,43.0,-5.0,-5.457538
1,"<a href=""https://emersoncollegepolling.com/jan...",2026-01-17,2026-01-19,1000 LV,1.422481e-01,48.1,41.7,-6.4,-7.864698
2,"<a href=""https://researchco.ca/2026/01/26/amen...",2026-01-13,2026-01-15,1002 A,7.429973e-02,42.0,36.0,-6.0,-5.324899
3,"<a href=""https://d3nkl3psvxxpe9.cloudfront.net...",2026-01-16,2026-01-19,1549 RV,7.027522e-02,43.0,39.0,-4.0,-4.129447
4,"<a href=""https://pro.morningconsult.com/tracke...",2026-01-12,2026-01-18,12505 RV,6.155048e-02,44.0,42.0,-2.0,-4.308718
...,...,...,...,...,...,...,...,...,...
270,"<a href=""https://napolitannews.org/posts/gener...",2025-01-15,2025-01-16,1000 RV,9.399664e-20,44.0,51.0,7.0,3.458351
271,"<a href=""https://mclaughlinonline.com/pols/wp-...",2025-01-22,2025-01-27,1000 LV,6.767335e-20,40.0,48.0,8.0,2.931762
272,"<a href=""https://quantusinsights.org/f/trump%E...",2025-01-22,2025-01-23,1000 RV,3.366666e-20,45.0,48.0,3.0,1.221893
273,"<a href=""https://www.cygn.al/poll-musk-most-ef...",2024-12-09,2024-12-11,1500 LV,2.723924e-21,44.6,48.4,3.8,2.789120


In [16]:
worksheet = spreadsheet.worksheet("generic_ballot_polls_display_table")
worksheet.clear()

set_with_dataframe(worksheet, gb_polls)